In [52]:
import pandas as pd
import sweetviz as sv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import warnings
import mlflow
import pickle
import category_encoders as ce
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.plots import plot_evaluations, plot_objective
from sklearn.model_selection import (
    StratifiedKFold,
    ShuffleSplit,
    KFold,
    cross_validate,
    cross_val_predict,
    GridSearchCV,
    train_test_split
)
from lightgbm import LGBMClassifier, early_stopping
from sklearn.feature_selection import RFE, SelectKBest, f_classif

In [53]:
df_train = pd.read_csv("Data/train_data.csv")

In [54]:
def data_cleaning(raw_data):
    #Replacing the small amount of null values in children and country columns with sensible values
    null_replacements = {"children": 0.0,"country": "Unknown"} 
    clean_data = raw_data.fillna(null_replacements) 
    #Dropping only row with room type L
    clean_data = clean_data[clean_data['assigned_room_type'] != 'L']
    #Replace error entries with nulls
    clean_data['market_segment'].replace('Undefined', np.nan, inplace=True)
    clean_data['distribution_channel'].replace('Undefined', np.nan, inplace=True)
    #Dropping bookings with no guests
    empty_bookings = (clean_data.children == 0) & (clean_data.adults == 0) & (clean_data.babies == 0) # There are no guests when these conditions are true
    clean_data = clean_data[~empty_bookings] 
    #Dropping all the duplicates
    clean_data = clean_data.drop_duplicates()
    clean_data = clean_data.reset_index(drop=True)
    return clean_data
df_train = data_cleaning(df_train)

def feature_engineering(df):
    # Add a boolean variable for whether or not an agent was used to book
    df['used_agent'] = np.where(df['agent'].isnull(),0,1)
    # Combining weekdays and weekends for a total nights value
    df['total_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']
    # Adding a column to map hotel to 0/1
    df['hotel_binary'] = df['hotel'].map({'Resort Hotel': 0, 'City Hotel': 1})
    # Creating a flags for guests coming from Portugal and EU
    df['portugal_flag'] = (df['country'] == 'PRT').astype('float64')
    eu_countries = ['AUT, BEL, BGR, HRV, CYP, CZE, DNK, EST, FIN, FRA, DEU, GRC, HUN, IRL, ITA, LVA, LTU, LUX, MLT, NLD, POL']
    df['eu_flag'] = df['country'].isin(eu_countries).astype('float64')
    
    # Now finding the day of the week for each booking 

    # Dictionary to map the month to its corresponding number
    month_to_number = {
        "January": 1,
        "February": 2,
        "March": 3,
        "April": 4,
        "May": 5,
        "June": 6,
        "July": 7,
        "August": 8,
        "September": 9,
        "October": 10,
        "November": 11,
        "December": 12
    }
    # adding columns for the date of arrivaL
    df['arrival_date_year'] = df['arrival_date_year'].astype(str)
    df['arrival_date_month_number'] = df['arrival_date_month'].map(month_to_number).astype(str).str.zfill(2)  
    df['arrival_date_day_of_month'] = df['arrival_date_day_of_month'].astype(str).str.zfill(2)  
    # string in the format "YYYY-MM-DD"
    df['raw_date_string'] = df['arrival_date_year'] + "-" + \
                                    df['arrival_date_month_number'] + "-" + \
                                    df['arrival_date_day_of_month']
    df['raw_date_string']

    # Creating a function that transforms the date into its corresponding day of the week
    def find_day(raw_date_string):
        date_object = datetime.strptime(raw_date_string, "%Y-%m-%d")
        day_of_week = date_object.weekday()
        return day_of_week
    df['day_of_week_on_arrival'] = df['raw_date_string'].apply(find_day)

    # Adding a column to encode the seasonal affect of which week in the year the booking was made
    period = df['arrival_date_week_number'].nunique()
    two_pi = 2*np.pi
    df['sin_week'] = np.sin(two_pi * df['arrival_date_week_number'] / period)
    df['cos_week'] = np.cos(two_pi * df['arrival_date_week_number'] / period)
    
    df['arrival_date_day_of_month'] = df['arrival_date_day_of_month'].astype(int)
    df['arrival_date_month_number'] = df['arrival_date_month_number'].astype(int)
    
    return df
df_train = feature_engineering(df_train)

C:\Windows\Temp\ipykernel_24508\916740151.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean_data['market_segment'].replace('Undefined', np.nan, inplace=True)
C:\Windows\Temp\ipykernel_24508\916740151.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doin

In [55]:
numeric_feats = [
    "lead_time",
    "arrival_date_week_number",
    "arrival_date_day_of_month",
    "stays_in_weekend_nights",
    "stays_in_week_nights",
    "adults",
    "children",
    "babies",
    "previous_cancellations",
    'previous_bookings_not_canceled',
    "days_in_waiting_list",
    "adr",
    "required_car_parking_spaces",
    "total_of_special_requests",
    "total_nights",
    "day_of_week_on_arrival",
    "sin_week",
    "cos_week"
]

binary_feats = [
    "is_repeated_guest",
    "portugal_flag",
    "eu_flag",
    "used_agent",
    "hotel_binary",
]

categorical_feats = [
    "arrival_date_month",
    "meal",
    "country",
    "market_segment",
    "distribution_channel",
    "reserved_room_type",
    "assigned_room_type", 
    "deposit_type",
    "customer_type"
]

all_feats = numeric_feats + binary_feats + categorical_feats 

print(f"Number of features: {len(all_feats)}")

Number of features: 32


In [56]:
prep = ColumnTransformer(
    transformers = [
        ('pass', 'passthrough', numeric_feats + binary_feats),
        ('ordinal', ce.OrdinalEncoder(handle_missing="return_nan"), categorical_feats)
    ],
    remainder="drop",
    verbose_feature_names_out=False
).set_output(transform="pandas")

prep.fit(df_train, df_train['is_canceled'])

ColumnTransformer(transformers=[('pass', 'passthrough',
                                 ['lead_time', 'arrival_date_week_number',
                                  'arrival_date_day_of_month',
                                  'stays_in_weekend_nights',
                                  'stays_in_week_nights', 'adults', 'children',
                                  'babies', 'previous_cancellations',
                                  'previous_bookings_not_canceled',
                                  'days_in_waiting_list', 'adr',
                                  'required_car_parking_spaces',
                                  'total_of_special_requests', 'tota...
                                  'day_of_week_on_arrival', 'sin_week',
                                  'cos_week', 'is_repeated_guest',
                                  'portugal_flag', 'eu_flag', 'used_agent',
                                  'hotel_binary']),
                                ('ordinal',
                                 OrdinalEncoder(handle_missing='return_nan'),
                                 ['arrival_date_month', 'meal', 'country',
                                  'market_segment', 'distribution_channel',
                                  'reserved_room_type', 'assigned_room_type',
                                  'deposit_type', 'customer_type'])],
                  verbose_feature_names_out=False)

In [57]:
# Define the LGBM model
lgbm_model = LGBMClassifier(
    objective="binary",
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=5,
    num_leaves=100,
    reg_alpha=0.1,
    reg_lambda=0.1,
    min_split_gain=0.01,
    ming_child_weight=0.1,
    subsample=1.0,
    colsample_bytree=1.0,
    cat_smooth=1.0,
    random_state=42,
    verbose=-1
)

# Define the pipeline
lgbm_pipe = Pipeline([
    ("prep", clone(prep)),  # Assuming 'prep' is defined elsewhere
    ("model", clone(lgbm_model))
])

lgbm_pipe


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('pass', 'passthrough',
                                                  ['lead_time',
                                                   'arrival_date_week_number',
                                                   'arrival_date_day_of_month',
                                                   'stays_in_weekend_nights',
                                                   'stays_in_week_nights',
                                                   'adults', 'children',
                                                   'babies',
                                                   'previous_cancellations',
                                                   'previous_bookings_not_canceled',
                                                   'days_in_waiting_list',
                                                   'adr',
                                                   'required_car_parking_spaces',
                                                   'total_of...
                                                   'country', 'market_segment',
                                                   'distribution_channel',
                                                   'reserved_room_type',
                                                   'assigned_room_type',
                                                   'deposit_type',
                                                   'customer_type'])],
                                   verbose_feature_names_out=False)),
                ('model',
                 LGBMClassifier(cat_smooth=1.0, max_depth=5,
                                min_split_gain=0.01, ming_child_weight=0.1,
                                n_estimators=1000, num_leaves=100,
                                objective='binary', random_state=42,
                                reg_alpha=0.1, reg_lambda=0.1, verbose=-1))])

In [58]:
lgbm_pipe['prep'].fit_transform(df_train)['arrival_date_day_of_month']

0         6
1        23
2         1
3        30
4         3
         ..
90596    16
90597    16
90598    12
90599     7
90600     4
Name: arrival_date_day_of_month, Length: 90601, dtype: int32

In [59]:
fit_params = {"model__categorical_feature": categorical_feats}
scoring = ["neg_log_loss", "accuracy", "precision", "recall", "f1", "roc_auc"]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Since the pipeline drops the target, just pass df_train as it is
cv_results = cross_validate(
    clone(lgbm_pipe),
    df_train,
    df_train["is_canceled"],  # Target
    cv=kf,
    fit_params=fit_params,
    return_train_score=True,
    scoring=scoring
)

# Create a dictionary for the results
entry = {
    "model": "BaseModel",
    "neg_log_loss": np.mean(cv_results['test_neg_log_loss']),
    "accuracy": np.mean(cv_results['test_accuracy']),
    "precision": np.mean(cv_results['test_precision']),
    "recall": np.mean(cv_results['test_recall']),
    "f1": np.mean(cv_results['test_f1']),
    "roc_auc": np.mean(cv_results['test_roc_auc'])
}

# Convert to a DataFrame and output
results = pd.DataFrame([entry])
results

c:\Users\Store\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(


,model,neg_log_loss,accuracy,precision,recall,f1,roc_auc
0,BaseModel,-0.277443,0.875145,0.851023,0.802635,0.82611,0.946783


In [60]:
ss = ShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
cv_train_idx, cv_val_idx = next(ss.split(df_train))
df_train_cv = df_train.iloc[cv_train_idx]
df_val_cv = df_train.iloc[cv_val_idx]

assert len(set(df_train_cv.index) & set(df_val_cv.index)) == 0 

In [61]:
prep_cv = clone(prep)
X_train_cv = prep_cv.fit_transform(df_train_cv)
X_val_cv = prep_cv.transform(df_val_cv) 

In [62]:
fit_params = {
    "model__eval_set": [(X_val_cv, df_val_cv["is_canceled"])],
    "model__categorical_feature": categorical_feats,
    "model__callbacks": [early_stopping(30, first_metric_only=True)]
    }

In [63]:
bayes_space = [
    Integer(3, 15, name='max_depth'),
    Integer(32, 512, name='num_leaves'),
    Real(0.0, 100, name='reg_alpha'),
    Real(0.0, 100, name='reg_lambda'),
    Real(0.0, 10, name='min_split_gain'),
    Real(0.01, 1000, prior='log-uniform', name='min_child_weight'),
    Real(0.5, 1, name='subsample'),
    Real(0.5, 1, name='colsample_bytree'),
    Real(0.01, 150, name='cat_smooth'),
    Real(0.001, 0.1, prior='log-uniform', name='learning_rate'),
    Integer(5, 100, name='min_child_samples')
]

cv_params = {
    "max_depth": 5,
    "num_leaves": 100,
    "reg_alpha": 0.1,
    "reg_lambda": 0.1,
    "min_split_gain": 0.01,
    "min_child_weight": 0.1,  
    "subsample": 1.0,
    "colsample_bytree": 1.0,
    "cat_smooth": 1.0,
    "learning_rate": 0.1,
    "min_child_samples": 20,  
    "random_state": 42,
    "verbose": -1 
}

In [64]:
early_stopping_data = []

@use_named_args(bayes_space)
def objective(**params):
    global early_stopping_data

    print(f"Evaluating params: {params}")
    cv_params_i = cv_params.copy()
    for key, value in params.items():
        cv_params_i[key] = value

    lgbm_pipe['model'].set_params(**cv_params_i)

    lgbm_cv_results = cross_validate(
        lgbm_pipe,
        df_train_cv,
        df_train_cv["is_canceled"],
        cv=kf,
        fit_params=fit_params,
        return_estimator=True,
        return_train_score=True,
        scoring='neg_log_loss',
        error_score='raise'  # Raise errors directly
    )

    cv_models = [estimator['model'] for estimator in lgbm_cv_results['estimator']]
    # Collect best_iteration_ from each model
    best_iterations = [model.best_iteration_ for model in cv_models]

    # Append the median best_iteration_ to the global list
    median_best_iteration = int(np.median(best_iterations))
    early_stopping_data.append(median_best_iteration)

    metric_to_min = -np.mean(lgbm_cv_results['test_score'])
    print(f"Mean Negative Log Loss: {metric_to_min}")
    return metric_to_min

In [65]:
res_gp = gp_minimize(
    objective,
    bayes_space,
    n_initial_points=30,
    n_calls=100,
    random_state=42,
    verbose=False
)

Evaluating params: {'max_depth': 13, 'num_leaves': 120, 'reg_alpha': 77.96910002727695, 'reg_lambda': 59.68501579464871, 'min_split_gain': 4.4583275285359125, 'min_child_weight': 0.03161364550529615, 'subsample': 0.7296244459829336, 'colsample_bytree': 0.666854305569511, 'cat_smooth': 21.438594020111903, 'learning_rate': 0.020034427927560734, 'min_child_samples': 10}


c:\Users\Store\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[467]	valid_0's binary_logloss: 0.353949
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[3]	valid_0's binary_logloss: 0.654842
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[120]	valid_0's binary_logloss: 0.483782
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[464]	valid_0's binary_logloss: 0.355542
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[168]	valid_0's binary_logloss: 0.466395
Evaluated only: binary_logloss
Mean Negative Log Loss: 0.4190563032272417
Evaluating params: {'max_depth': 12, 'num_leaves': 483, 'reg_alpha': 0.07787658410143286, 'reg_lambda': 99.221155929121

c:\Users\Store\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.343972
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[27]	valid_0's binary_logloss: 0.654853
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[398]	valid_0's binary_logloss: 0.460672
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.347856
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[421]	valid_0's binary_logloss: 0.455374
Evaluated only: binary_logloss
Mean Negative Log Loss: 0.4113519095619248
Evaluating params: {'max_depth': 15, 'num_leaves': 144, 'reg_alpha': 9.060643453282081, 'r

...

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[311]	valid_0's binary_logloss: 0.327547
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[9]	valid_0's binary_logloss: 0.606105
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[31]	valid_0's binary_logloss: 0.472871
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[296]	valid_0's binary_logloss: 0.331432
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[41]	valid_0's binary_logloss: 0.444862
Evaluated only: binary_logloss
Mean Negative Log Loss: 0.3771272652224652
Evaluating params: {'max_depth': 13, 'num_leaves': 509, 'reg_alpha': 5.603355730448269, 'reg_lambda': 85.1218096733316, '

c:\Users\Store\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.44733
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[9]	valid_0's binary_logloss: 0.6578
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[999]	valid_0's binary_logloss: 0.502492
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.448891
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[999]	valid_0's binary_logloss: 0.491045
Evaluated only: binary_logloss
Mean Negative Log Loss: 0.48726185741344324
Evaluating params: {'max_depth': 3, 'num_leaves': 481, 'reg_alpha':

c:\Users\Store\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.395623
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[5]	valid_0's binary_logloss: 0.654682
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[333]	valid_0's binary_logloss: 0.499688
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.395367
Evaluated only: binary_logloss
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[369]	valid_0's binary_logloss: 0.478586
Evaluated only: binary_logloss
Mean Negative Log Loss: 0.4505273315962393


In [66]:
early_stopping_data

[168,
 421,
 273,
 687,
 47,
 110,
 86,
 144,
 1000,
 65,
 410,
 311,
 99,
 177,
 40,
 999,
 787,
 119,
 61,
 108,
 258,
 329,
 1000,
 555,
 1000,
 113,
 47,
 195,
 39,
 1000,
 41,
 82,
 80,
 47,
 503,
 79,
 57,
 41,
 41,
 31,
 25,
 32,
 1000,
 31,
 32,
 41,
 31,
 47,
 31,
 31,
 117,
 17,
 31,
 31,
 31,
 40,
 31,
 31,
 122,
 580,
 31,
 31,
 24,
 259,
 65,
 171,
 112,
 47,
 47,
 142,
 31,
 32,
 31,
 153,
 31,
 31,
 31,
 54,
 31,
 82,
 31,
 23,
 31,
 31,
 31,
 1000,
 21,
 47,
 54,
 142,
 1000,
 350,
 47,
 53,
 264,
 548,
 250,
 41,
 999,
 369]

In [79]:
bayes_best_iter = np.argmin(res_gp.func_vals)
best_iter_early_stopping = early_stopping_data[bayes_best_iter]
n_estimators_refit = best_iter_early_stopping
print(f"n_estimators for Refit: {n_estimators_refit}")


ayes_best_score = res_gp.fun
bayes_best_hps = res_gp.x
bayes_best_score, bayes_best_hps

n_estimators for Refit: 31


(0.3082364836289463,
 [15, 512, 0.0, 0.0, 0.0, 0.04664053990862526, 1.0, 0.5, 0.01, 0.1, 5])

In [69]:
bayes_best_params = {res_gp.space[i][1].name: res_gp.x[i] for i in range(len(res_gp.x))}
bayes_best_params

{'max_depth': 15,
 'num_leaves': 512,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'min_split_gain': 0.0,
 'min_child_weight': 0.04664053990862526,
 'subsample': 1.0,
 'colsample_bytree': 0.5,
 'cat_smooth': 0.01,
 'learning_rate': 0.1,
 'min_child_samples': 5}

In [82]:
# Update the best parameters with the number of estimators for refit
bayes_best_params.update({"n_estimators": 1000})

# Define the LGBM model with the best parameters
lgbm_best_model = LGBMClassifier(**bayes_best_params)

# Fit the model on the entire training set
lgbm_best_model.fit(X_train_cv, df_train_cv["is_canceled"])

# Define the pipeline
lgbm_pipe_best = Pipeline([
    ("prep", clone(prep)),  
    ("model", clone(lgbm_best_model))
])

In [83]:

fit_params = {"model__categorical_feature": categorical_feats}
scoring = ["neg_log_loss", "accuracy", "precision", "recall", "f1", "roc_auc"]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Since the pipeline drops the target, just pass df_train as it is
cv_results = cross_validate(
    clone(lgbm_pipe_best),
    df_train.drop(columns=["is_canceled"]),
    df_train["is_canceled"],  # Target
    cv=kf,
    fit_params=fit_params,
    return_train_score=True,
    scoring=scoring
)

# Create a dictionary for the results
entry = {
    "model": "BayesOptModel",
    "neg_log_loss": np.mean(cv_results['test_neg_log_loss']),
    "accuracy": np.mean(cv_results['test_accuracy']),
    "precision": np.mean(cv_results['test_precision']),
    "recall": np.mean(cv_results['test_recall']),
    "f1": np.mean(cv_results['test_f1']),
    "roc_auc": np.mean(cv_results['test_roc_auc'])
}
entry = pd.DataFrame([entry])

# Concatenate the new entry to the results DataFrame
results = pd.concat([results, entry], ignore_index=True)
results

c:\Users\Store\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(


,model,neg_log_loss,accuracy,precision,recall,f1,roc_auc
0,BaseModel,-0.277443,0.875145,0.851023,0.802635,0.826110,0.946783
1,BayesOptModel,-0.304009,0.871392,0.877105,0.758179,0.813303,0.944972
2,BayesOptModel,-0.379903,0.822232,0.802905,0.687782,0.740882,0.898740
3,BayesOptModel,-0.318935,0.885310,0.869565,0.811327,0.839433,0.953486
4,BayesOptModel,-0.420240,0.884317,0.865784,0.812975,0.838541,0.952324
